In [ ]:
# ===== MODEL COMPARISON UTILITY =====
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

def run_and_eval(model_name, model_obj, X_tr, y_tr_s, X_va, y_va, y_scaler):
    """Fit, predict, inverse scale, and return metrics DataFrame + overall scores."""
    model = MultiOutputRegressor(model_obj, n_jobs=-1)
    model.fit(X_tr, y_tr_s)
    y_pred_s = model.predict(X_va)
    y_pred = y_scaler.inverse_transform(y_pred_s)
    per_target, overall = metrics_table(y_va, y_pred)
    per_target["Model"] = model_name
    return per_target, overall

# Example models to compare
models_to_compare = {
    "XGB": make_xgb(RANDOM_STATE),
    "RandomForest": RandomForestRegressor(
        n_estimators=300, max_depth=12, n_jobs=-1, random_state=RANDOM_STATE
    ),
    "SVR": SVR(kernel="rbf", C=5.0, epsilon=0.1)  # will be wrapped in MultiOutputRegressor
}

# Run comparison
comparison_rows = []
overall_rows = []

for name, mdl in models_to_compare.items():
    print(f"\n[INFO] Training {name}...")
    per_target_df, overall_scores = run_and_eval(name, mdl, X_tr, y_tr_s, X_va, y_va, y_scaler)
    comparison_rows.append(per_target_df)
    overall_rows.append({"Model": name, **overall_scores})

# Combine results
comparison_df = pd.concat(comparison_rows, axis=0)
overall_df = pd.DataFrame(overall_rows)

print("\n=== Overall Scores ===")
print(overall_df)

# ===== VISUAL COMPARISON =====
import seaborn as sns
plt.figure(figsize=(12, 6))
sns.barplot(
    data=comparison_df,
    x="Target", y="R2", hue="Model",
    palette="Set2"
)
plt.xticks(rotation=45, ha="right")
plt.ylabel("R² (Validation Set)")
plt.title("Per-Target R² Comparison Across Models", weight="bold")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.ylim(0.0, 1.0)
plt.legend(title="Model")
plt.tight_layout()
plt.show()
